# Section 3: The Pangeo Approach - An Implementation Guide

## Cloud computing

Info about cloud providers
Description oif core services
* compute instances
* storage
* ??

## Containers

introduction top docker
how to build a container
what goes into a container
deploying a container

## orchestration  - kubernetes

Intro to kubernetes

demo code: creating ac cluster

## distribution - Dask

introduction to dask

discussion of task graph and data level parallelism



In [1]:
# Demo code: doing some analysis with a dask kubernetes cluster on a pangeo instance
# we will actually need a pangeo instance to demo this

## Interactivity & portability - Jupyter Labs

Introduction to Jupyer labs

(use material from pythobn guild)

In [ ]:
#demo running compute (again)
#demo accessing data in notebook
# demo

## Dashboarding

What is a dashboard
introduction to bokeh for interactive visualisation



In [2]:
# demo bokeh, holoview in a jupyter notebook